In [2]:
%load_ext autoreload
%autoreload 2

In [45]:
from util import *
from extract_direct_indicators import *
from extract_indirect_indicators import *
from extract_time_indicator import *

### Direct Indicator

In [57]:
dir_ind = extract_direct_indicators(sidewalk_qualifer, 'rec')
dir_ind = dir_ind[dir_ind['visibility'] == 'V'] # Exclude deleted or changed entries
stats = get_stats_dir(dir_ind)
# Using ceil(mean) as central tendency measure
stats = stats.loc['mean'].apply(np.ceil).to_dict()

In [58]:
def compute_dir(**kwargs):
    res = dict()
    for key in kwargs:
        res[key]  = 1 if kwargs[key] >= stats[key] else 0

    Wnum, Wdir_c, Wuc, Wedit, WrollB, WTag = 0.2, 0.2, 0.2, 0.1, 0.1, 0.2
    Td = (Wnum * res['nversions']) + (Wdir_c * res['dir_confirmations']) + \
         (Wuc * res['nusers']) + (Wedit * res['nedits']) + \
         (WrollB * res['nrollbacks']) + (WTag * res['ntags'])

    return Td

dir_ind['Td'] = dir_ind.apply(lambda row: compute_dir(
                                                    nversions = row.nversions, 
                                                    dir_confirmations = row.dir_confirmations, 
                                                    nusers = row.nusers, 
                                                    nedits = row.nedits, 
                                                    nrollbacks = row.nrollbacks, 
                                                    ntags = row.ntags
                                                    ), 
                                                    axis=1)

### Time Indicator

In [63]:
time_ind = extract_time_indicator(sidewalk_qualifer, 'rec')
stats = get_stats(time_ind)
avg = stats.loc['mean'].apply(np.ceil)[0] # Average last edit
time_ind['Ttime'] = time_ind.apply(lambda row: 1 if row['last_edit(months)'] >= avg else 0, axis=1)

In [52]:
time_ind

,id,ts,last_edit(months)
0,379240246,2020-09-15 05:46:49+00:00,23
1,379240248,2021-07-20 23:16:22+00:00,13
2,482849683,2019-08-01 17:15:19+00:00,36
3,483240568,2020-02-11 01:24:12+00:00,30
4,491692399,2017-05-05 23:54:22+00:00,63
...,...,...,...
269,1035042936,2022-02-25 11:15:29+00:00,6
270,1035042938,2022-02-25 11:15:29+00:00,6
271,1035042941,2022-02-25 11:15:29+00:00,6
272,1035042942,2022-02-25 11:15:29+00:00,6


In [ ]:
# T(F) = Td(F).Wd + Ti(F).Wi + Ttime(F).Wtime
# Wd = 0.5
# Wi = 0.25
# Wtime = 0.25

# Td(F) = (Wnum.V_num) + (Wdir_c.Dir_C) + (Wuc.UC) + (Wedit.Tag_edits) + (WrollB.RollBk) + (WTag.Tag)
# Wnum = 0.2
# Wdir_c = 0.2
# Wuc = 0.2
# Wedit = 0.1
# WrollB = 0.1
# WTag = 0.2

In [ ]:
# To measure trustworthiness of sidewalk data: dir/indir indicators of the *road* that is associated with that sidewalk!

# Two steps:

# Check if sidewalk-data availabile or not. Four cases:
    # i) Partial availability (Case 1): sidewalk geometry available but tag is not available on the associated road
    # ii) Partial availability (Case 2): sidewalk geometry not availble, but sidewalk tag (yes/ left/ right/ both) is on the road
    # iii) Full availibility: Both sidewalk geometry and sidewalk tag on the associated road are available. Or, if sidewalk is not there then tag is (no/none)
    # iv) Unavailable: Neither geometry nor the tag of associated road is available

# if available, check if the associated road data is trustworthy or not

In [ ]:
# Change indirect indicator for sidewalks
# previously poi, buildings, roads and had lists of buildings per cell
# Now measure: poi, buildings, roads(except sidewalks) and sidewalks. Then compute cells for sidewalks with ids. Finally for each sidewalk compute indir ind

# feature availability for sidewalk:left, sidewalk:right, sidewalk:both
# If sidewalk features are also and mostly available on associated roads, then we have to change approach. Discuss with professor!!

In [ ]:
# Indirect and time indicator measurable
# But direct indicator not so as sidewalk data only started to increase in recent times and barely have 2 tags only!! Still have to explore more for it

In [ ]:
# Instead of storing the pbf files for history o rlatest osm
# extract and store nodes, ways, relations (for latest osm data)
# and also other important attributes required (required for dir/indir indicators) for hisotrical data 
# create dataframes and store them in feather files!!

# And then extract them for direct or indirect indicators and use accordingly (Have to change a lot of code for that!!)